In [16]:
import torch

In [38]:
text="我来到了北京清华大学。"

In [18]:
#测试一下FMM/BMM
from algorithm.FMM_BMM import FMM_BMM
import os
from utils.utils import load_dictionary
dataset_path = './dataset/icwb2-data'
training_file = os.path.join(dataset_path, 'gold/msr_training_words.utf8')
dictionary = load_dictionary(training_file)
fmm_bmm = FMM_BMM(dictionary)

fmm = ' '.join(fmm_bmm.forward_max_matching(text))
bmm = ' '.join(fmm_bmm.backward_max_matching(text))

print(fmm)
print(bmm)

我 来到 了 北京 清华大学 。
我 来到 了 北京 清华大学 。


In [19]:
#测试一下 HMM
from algorithm.HMM import HMMTokenizer
hmm_tokenizer = HMMTokenizer()
model_dir = "/home/shaoxiong/exa/人工智能大作业/algorithm/model/HMM"
hmm_tokenizer.load_model(model_dir)

hmm = ' '.join(hmm_tokenizer.tokenize(text))
print(hmm)

我来 到 了 北京 清华 大学 。


In [20]:
#测试一下N-gram
from algorithm.NGram import NgramTokenizer
ngram_tokenizer = NgramTokenizer()
model_dir = "/home/shaoxiong/exa/人工智能大作业/algorithm/model/N-Gram"
ngram_tokenizer.load_model(model_dir)

ngram = ' '.join(ngram_tokenizer.tokenize(text))
print(ngram)


我来到 了北京 清华大学。


In [42]:
#测试一下RNN
from algorithm.RNN import RNNSegmentation
import torch
class Vocabulary:
    def __init__(self):
        self.char_to_idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx_to_char = {0: '<PAD>', 1: '<UNK>'}
        self.tag_to_idx = {'<PAD>': 0, 'B': 1, 'M': 2, 'E': 3, 'S': 4}
        self.idx_to_tag = {0: '<PAD>', 1: 'B', 2: 'M', 3: 'E', 4: 'S'}
        
    def add_character(self, char: str):
        if char not in self.char_to_idx:
            idx = len(self.char_to_idx)
            self.char_to_idx[char] = idx
            self.idx_to_char[idx] = char
            
    def get_vocab_size(self) -> int:
        return len(self.char_to_idx)
    
    def get_tag_size(self) -> int:
        return len(self.tag_to_idx)
vocab = Vocabulary()
rnn_model = RNNSegmentation(5169, 
                          vocab.get_tag_size(),
                          embedding_dim=128,
                          hidden_dim=256)
ckpj = torch.load("/home/shaoxiong/exa/人工智能大作业/algorithm/model/RNN-based/rnn_segmentation_model.pt")
rnn_model.load_state_dict(ckpj['model_state_dict'])
rnn_model.eval()
from algorithm.RNN import segment_text
rnn = ' '.join(segment_text(rnn_model.to('cuda'), text, vocab))
print(rnn)

我们 班 的 同 学 都 到 了 学 校 。


In [41]:
#测试BiLSTM
from algorithm.BiLSTM import BiLSTMSegmentation,segment_text
bilstm_model = BiLSTMSegmentation(5169, 
                             vocab.get_tag_size(),
                             embedding_dim=256,  # 增大embedding维度
                             hidden_dim=512)
ckpj = torch.load("/home/shaoxiong/exa/人工智能大作业/algorithm/model/BiLSTM-based/bilstm_segmentation_model.pt")
bilstm_model.load_state_dict(ckpj['model_state_dict'])
bilstm_model.eval()
bilstm = ' '.join(segment_text(bilstm_model.to('cuda'), text, vocab))
print(bilstm)

我们 班 的 同学 都 到 了 学校 。


In [40]:
#测试Transformers
from algorithm.Transformers import ChineseSegmentationTransformer,segment_text

ckpj = torch.load("/home/shaoxiong/exa/人工智能大作业/algorithm/model/Transformers-based/chinese_segmentation_transformer.pt")
vocab = ckpj['vocab']
model_config = ckpj['model_config']
transformer_model = ChineseSegmentationTransformer(
        vocab_size=vocab.get_vocab_size(),
        tag_size=vocab.get_tag_size(),
        d_model=128,  # 减小模型维度
        num_heads=4,  # 减少注意力头数
        num_layers=4,  # 减少层数
        d_ff=512,     # 减小前馈网络维度
        dropout=0.2   # 增加dropout
    )
transformer_model.load_state_dict(ckpj['model_state_dict'])
transformer_model.eval()
text="我们班的同学都到了学校。"
transformer = ' '.join(segment_text(transformer_model.to('cuda'), text, vocab))
print(transformer)

我们 班的 同 学都 到 了 学校 。
